E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [1]:
words = open('../names.txt', 'r').read().splitlines()

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

### Tiagram Data Models

In [17]:
t = {}
for w in words:
    chars = ["."] + list(w) + ["."]
    for ch1,ch2,ch3 in zip(chars,chars[1:],chars[2:]):
        tiagram = (ch1,ch2,ch3)
        t[tiagram] = t.get(tiagram,0)+1

In [19]:
t

{('.', 'e', 'm'): 288,
 ('e', 'm', 'm'): 100,
 ('m', 'm', 'a'): 72,
 ('m', 'a', '.'): 174,
 ('.', 'o', 'l'): 104,
 ('o', 'l', 'i'): 69,
 ('l', 'i', 'v'): 54,
 ('i', 'v', 'i'): 78,
 ('v', 'i', 'a'): 147,
 ('i', 'a', '.'): 903,
 ('.', 'a', 'v'): 243,
 ('a', 'v', 'a'): 161,
 ('v', 'a', '.'): 93,
 ('.', 'i', 's'): 124,
 ('i', 's', 'a'): 142,
 ('s', 'a', 'b'): 76,
 ('a', 'b', 'e'): 173,
 ('b', 'e', 'l'): 201,
 ('e', 'l', 'l'): 822,
 ('l', 'l', 'a'): 337,
 ('l', 'a', '.'): 684,
 ('.', 's', 'o'): 152,
 ('s', 'o', 'p'): 21,
 ('o', 'p', 'h'): 37,
 ('p', 'h', 'i'): 61,
 ('h', 'i', 'a'): 81,
 ('.', 'c', 'h'): 352,
 ('c', 'h', 'a'): 236,
 ('h', 'a', 'r'): 329,
 ('a', 'r', 'l'): 287,
 ('r', 'l', 'o'): 44,
 ('l', 'o', 't'): 14,
 ('o', 't', 't'): 34,
 ('t', 't', 'e'): 121,
 ('t', 'e', '.'): 175,
 ('.', 'm', 'i'): 393,
 ('m', 'i', 'a'): 95,
 ('.', 'a', 'm'): 384,
 ('a', 'm', 'e'): 226,
 ('m', 'e', 'l'): 188,
 ('e', 'l', 'i'): 537,
 ('l', 'i', 'a'): 518,
 ('.', 'h', 'a'): 505,
 ('a', 'r', 'p'): 8,
 ('r

In [13]:
chars = sorted(list((set(''.join(words)))))

In [14]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0;
itos = {i:s for s,i in stoi.items()}

### Couting approach

In [15]:
import torch

In [20]:
N = torch.zeros((27,27,27),dtype = torch.int32)
N

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [27]:
N.shape

torch.Size([27, 27, 27])

In [21]:
for w in words:
    chars = ["."] + list(w) + ["."]
    for ch1,ch2,ch3 in zip(chars,chars[1:],chars[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        N[ix1,ix2,ix3] += 1
        

In [33]:
P = (N+2).float()
P /= P.sum(2,keepdim=True)

In [34]:
P.shape

torch.Size([27, 27, 27])

In [37]:
g = torch.Generator().manual_seed(2147483647)
for i in range(10):
    out = []
    ix1 = 0;
    ix2 = 0;
    while True:
        p = P[ix1][ix2]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break;
        else:
            ix1 = ix2
            ix2 = ix
    print(''.join(out))

junide.
ilyasid.
prelay.
ocin.
fai.
ritoper.
sathen.
dannaaryanileniassibduinrwin.
lessiyanayla.
te.


In [38]:
log_likelihood = 0.0
n = 0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        prob = P[ix1,ix2,ix3]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        # print(f'{ch1}{ch2} : {prob:.4f} {logprob: .4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')

log_likelihood=tensor(-414938.1562)
nll=tensor(414938.1562)
nll/n=tensor(2.1158)


### Neural Network Approach

In [50]:
xs , ys = [],[]
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        print(ch1,ch2,ch3)
        xs.append((ix1,ix2))
        ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

. e m
e m m
m m a
m a .


In [51]:
xs

tensor([[ 0,  5],
        [ 5, 13],
        [13, 13],
        [13,  1]])

In [52]:
ys

tensor([13, 13,  1,  0])

In [59]:
import torch.nn.functional as F
xenc = F.one_hot(xs,num_classes=27).float()
xenc = xenc.view(-1,27*2)

In [60]:
xenc[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [61]:
xenc.shape

torch.Size([4, 54])

In [63]:
W = torch.randn((27*2,1))
xenc @ W

tensor([[1.1631],
        [0.0642],
        [0.1443],
        [1.0912]])

In [64]:
W = torch.randn((27*2,27))
xenc @ W 

tensor([[-0.6337, -0.0385,  0.9050, -0.4025,  1.9496, -0.4164,  1.9554, -0.3180,
          1.3450,  2.7747, -1.2157,  0.7114,  0.1034, -0.4844,  0.5300,  0.0088,
          1.1168, -1.6547,  0.8506, -0.3570, -1.8706, -0.9108,  1.5011, -0.1444,
          1.0888,  1.0879,  0.0905],
        [ 2.0191,  1.1537,  1.8008,  0.8533, -0.0615,  0.1624,  0.7575,  1.3243,
          1.6254, -0.2394, -1.7812, -2.9817, -1.2129, -0.2124,  1.6122,  1.0449,
         -0.9000, -3.0156, -0.4307,  0.8167, -0.0301,  0.4327, -1.7049, -2.4585,
          0.1511, -2.1047, -1.0600],
        [ 0.8878, -0.3797,  2.3195, -0.1073,  0.5469,  2.1882,  1.1913,  1.4039,
          0.6252,  0.7494, -1.3530, -3.3414, -0.4538,  0.4707,  1.4602,  0.9559,
         -0.3903, -3.4806, -1.3603, -0.2296,  2.2159,  1.8876, -0.8554, -2.6051,
          0.1267, -1.0469, -0.8861],
        [-0.6410, -0.0060,  2.5177,  1.0860,  0.7425,  1.0379,  0.0846,  0.9844,
          0.0277,  1.0019, -1.4922,  1.5161,  0.7434, -0.0595,  1.7985, -0.7157

In [66]:
(xenc @ W)[0,2] # firing rate of second neuron looking at zero input

tensor(0.9050)

In [67]:
xenc[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [68]:
W[:2]

tensor([[ 0.0173,  1.0620,  0.2742, -0.6521,  1.7980, -0.7365,  1.4010,  1.3797,
          0.9673,  1.1327,  0.1981,  0.2406, -1.2343, -0.5934,  1.4536,  0.0510,
          1.2298, -0.8402,  0.2647, -0.6073, -0.4084,  0.3185,  1.6431,  0.5498,
          1.6268,  0.5219,  0.7061],
        [ 0.1266,  0.7558, -0.0928, -1.9752, -1.0777,  1.1795, -0.1773,  0.4624,
         -2.6492,  0.9517,  0.3174,  2.3714,  0.3216, -0.4365, -0.1587, -2.3094,
          0.2215,  0.8469,  0.2972,  0.2207, -0.2670,  0.4881,  0.3472,  0.1795,
         -0.7712, -0.7954, -1.1595]])

In [70]:
(xenc[0] * W[:,2]).sum()

tensor(0.9050)